## Avaliando [Crash Course](https://mxnet.apache.org/versions/master/api/python/docs/tutorials/getting-started/crash-course/4-components.html)


In [1]:
from mxnet import np,npx,gluon
import mxnet as mx
from mxnet.gluon import nn, loss as gloss
from mxnet.gluon.metric import Accuracy

npx.set_np()

device=mx.cpu()

In [2]:
net=nn.Sequential()

net.add(nn.Dense(5,in_units=3,activation='relu'),
        nn.Dense(25,activation='relu'),
        nn.Dense(2)
)

net

Sequential(
  (0): Dense(3 -> 5, Activation(relu))
  (1): Dense(-1 -> 25, Activation(relu))
  (2): Dense(-1 -> 2, linear)
)

In [3]:
net.initialize()
params=net.collect_params()

for key,value in params.items():
    print(key,value)

0.weight Parameter (shape=(5, 3), dtype=float32)
0.bias Parameter (shape=(5,), dtype=float32)
1.weight Parameter (shape=(25, -1), dtype=float32)
1.bias Parameter (shape=(25,), dtype=float32)
2.weight Parameter (shape=(2, -1), dtype=float32)
2.bias Parameter (shape=(2,), dtype=float32)


[21:35:17] /home/mertins/Desenvolvimento/C/Terceiros/mxnet/src/storage/storage.cc:202: Using Pooled (Naive) StorageManager for CPU


In [4]:
x=np.random.uniform(-1,1,(10,3))
net(x) #Forward computation

params=net.collect_params()
for key,value in params.items():
    print(key,value)

0.weight Parameter (shape=(5, 3), dtype=float32)
0.bias Parameter (shape=(5,), dtype=float32)
1.weight Parameter (shape=(25, 5), dtype=float32)
1.bias Parameter (shape=(25,), dtype=float32)
2.weight Parameter (shape=(2, 25), dtype=float32)
2.bias Parameter (shape=(2,), dtype=float32)


#### pesos inicializados na primeira camada

In [5]:
print(net[0].weight.data()[0])
print(net[0].weight.data()[1])
print(net[0].weight.data()[2])
print(net[0].weight.data()[3])
print(net[0].weight.data()[4])

[0.0068339  0.01299825 0.0301265 ]
[0.04819721 0.01438687 0.05011239]
[ 0.00628365  0.04861524 -0.01068833]
[ 0.01729892  0.02042518 -0.01618656]
[-0.00873779 -0.02834515  0.05484822]


### Build-in Initialization

In [6]:
from mxnet import init

net.initialize(init=init.Constant(3),device=device,force_reinit=True)
print(net[0].weight.data()[0])
print(net[0].weight.data()[1])
print(net[0].weight.data()[2])
print(net[0].weight.data()[3])
print(net[0].weight.data()[4])

[3. 3. 3.]
[3. 3. 3.]
[3. 3. 3.]
[3. 3. 3.]
[3. 3. 3.]


In [7]:
net.initialize(init=init.Normal(sigma=0.2),device=device,force_reinit=True)
print(net[0].weight.data()[0])
print(net[0].weight.data()[1])
print(net[0].weight.data()[2])
print(net[0].weight.data()[3])
print(net[0].weight.data()[4])

[-0.38397798 -0.3996415  -0.13511692]
[ 0.07022709  0.2015498  -0.15289484]
[-0.07078241  0.25188705  0.32647833]
[-0.1925947  -0.06670957  0.10161012]
[-0.34756812  0.15123469  0.15474765]


### Loss function

In [8]:
net=nn.Dense(1)
net.initialize()

nn_input=np.array([[1.2,0.56],
                    [3.0,0.72],
                    [0.89,0.9],
                    [0.89,2.3],
                    [0.99,0.52]])

nn_output=net (nn_input)
nn_output

array([[-0.05594432],
       [-0.12212431],
       [-0.05413363],
       [-0.09064993],
       [-0.04766688]])

In [9]:
temp=np.array([[0.0083],
               [0.00382],
               [0.02061],
               [0.00495],
               [0.00639]])
groundtruth_label=temp.reshape(5,1)
temp.shape,groundtruth_label.shape

((5, 1), (5, 1))

In [10]:
def L2Loss(output_value,true_values):
    return np.mean((output_value-true_values)**2,axis=1)/2

L2Loss(nn_output,groundtruth_label)

array([0.00206367, 0.00793099, 0.00279331, 0.00456967, 0.00146107])

In [11]:
loss=gloss.L2Loss()

loss(nn_output,groundtruth_label)

array([0.00206367, 0.00793099, 0.00279331, 0.00456967, 0.00146107])

In [12]:
class custom_L1_loss(gloss.Loss):
    def __init__(self, weight=None, batch_axis=0, **kwargs):
        super(custom_L1_loss,self).__init__(weight, batch_axis, **kwargs)
    def forward(self,pred,label):
        l=np.abs(label-pred)
        l=l.reshape(len(l),)
        return l
    
L1=custom_L1_loss()
L1(nn_output,groundtruth_label)

array([0.06424432, 0.12594432, 0.07474363, 0.09559993, 0.05405688])

In [13]:
l1=gloss.L1Loss()
l1(nn_output,groundtruth_label)

array([0.06424432, 0.12594432, 0.07474363, 0.09559993, 0.05405688])

### Optimizer

In [14]:
from mxnet import optimizer

In [15]:
trainer=gluon.Trainer(net.collect_params(),
                      optimizer="Adam",
                      optimizer_params={
                          "learning_rate":0.1,
                          "wd":0.001
                        })

In [16]:
curr_weight=net.weight.data()
print(curr_weight)

[[-0.03444817 -0.02608307]]


In [17]:
batch_size=len(nn_input)
trainer.step(batch_size,ignore_stale_grad=True)
print(net.weight.data())

[[-0.03444817 -0.02608307]]


In [18]:
print(curr_weight-net.weight.grad()*1/5)

[[-0.03444817 -0.02608307]]


### Metrics

In [19]:
pred=np.array([[0.1, 0.9], [0.05, 0.95], [0.83, 0.17], [0.63, 0.37]])

labels=np.array([1,1,0,1])

In [20]:
acc=Accuracy()

In [21]:
acc.update(labels=labels,preds=pred)

#### Creating custom metrics

In [22]:
def MyCustomMetric(EvalMetric):
    def __init__(self):
        super().init()
    def update(self,labels,preds):
        pass